In [1]:
import os
import string
import sys

import numpy as np
import pandas as pd

In [2]:
%store -r fire_zones
%store -r years
%store -r months

In [3]:
def get_data(df, path, zone):
    data = pd.read_csv(path)
    data = data[["Year", "Month", "Mean Temp (°C)", "Total Precip (mm)"]]
    data[["Zone"]] = zone
    df = pd.concat([df, data])

In [4]:
#code for iterating through directory from geeksforgeeks https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
weather_data = pd.DataFrame(columns = ["year", "Month", "Mean Temp (°C)", "Total Precip (mm)", "Zone"])
print(weather_data.head())
#for zone in fire_zones:
zone = "Cariboo_Quesnel"
directory = "data/HistoricalWeatherData/" + zone
for filename in os.listdir(directory):
    file = os.path.join(directory, filename)
    if os.path.isfile(file):
        get_data(weather_data, file, zone)

Empty DataFrame
Columns: [year, Month, Mean Temp (°C), Total Precip (mm), Zone]
Index: []


In [22]:
weather_data.to_csv("test.csv")